## 1. Configuration et Imports

In [ ]:
# Imports nécessaires
import sys
import os
import json
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta

# Ajouter le chemin du projet
project_root = os.path.dirname(os.getcwd())
sys.path.insert(0, project_root)

print("✅ Configuration terminée !")
print(f"📁 Projet racine : {project_root}")

## 2. Chargement des Composants du Chatbot

In [ ]:
# Charger les modules du chatbot
from src.approach1.sentiment_analyzer import SentimentAnalyzer
from src.approach1.mood_tracker import MoodTracker

# Initialiser les composants
print("🔄 Chargement des composants...")
analyzer = SentimentAnalyzer()
tracker = MoodTracker()
print("✅ Composants chargés !")

## 3. Exploration de l'Historique d'Humeur

In [ ]:
# Charger et afficher l'historique
history = tracker.mood_history

print(f"📊 Nombre total de messages : {len(history)}")

if history:
    # Convertir en DataFrame pour l'analyse
    df = pd.DataFrame(history)
    print("\n📋 Aperçu des données :")
    display(df.head(10))
    
    print("\n📊 Statistiques descriptives :")
    display(df.describe())
else:
    print("⚠️ Aucun historique disponible. Créons des données de test...")

## 4. Génération de Données de Test

In [ ]:
# Phrases de test pour l'analyse
test_phrases = [
    # Positif
    "Je suis vraiment heureux aujourd'hui !",
    "Quelle belle journée, je me sens en forme !",
    "J'ai réussi mon examen, c'est génial !",
    "Merci beaucoup, ça m'a fait du bien !",
    "Je me sens épanoui et confiant !",
    
    # Négatif
    "Je me sens triste et fatigué...",
    "Cette journée est vraiment difficile.",
    "Je suis stressé par le travail.",
    "Je me sens seul et incompris.",
    "Rien ne va comme je veux...",
    
    # Neutre
    "Il fait beau dehors.",
    "Je viens de me réveiller.",
    "C'est l'heure du déjeuner.",
    "J'ai lu un livre hier.",
    "Je vais faire des courses."
]

# Analyser toutes les phrases
results = []
for phrase in test_phrases:
    result = analyzer.analyze(phrase)
    results.append({
        'text': phrase,
        'sentiment': result['sentiment'],
        'confidence': result['confidence'],
        'predicted_class': result['predicted_class']
    })

# Créer le DataFrame
df_test = pd.DataFrame(results)
print(f"✅ {len(results)} phrases analysées !")
display(df_test)

## 5. Distribution des Sentiments

In [ ]:
# Graphique en camembert de la distribution
sentiment_counts = df_test['sentiment'].value_counts()

colors = {
    'très positif': '#4CAF50',
    'positif': '#8BC34A',
    'neutre': '#FFC107',
    'négatif': '#FF9800',
    'très négatif': '#F44336'
}

fig = px.pie(
    values=sentiment_counts.values,
    names=sentiment_counts.index,
    title='🎯 Distribution des Sentiments',
    color=sentiment_counts.index,
    color_discrete_map=colors,
    hole=0.4
)

fig.update_traces(
    textposition='inside',
    textinfo='percent+label'
)

fig.update_layout(
    font=dict(size=14),
    showlegend=True
)

fig.show()

## 6. Confiance par Sentiment

In [ ]:
# Box plot de la confiance par sentiment
fig = px.box(
    df_test,
    x='sentiment',
    y='confidence',
    color='sentiment',
    color_discrete_map=colors,
    title='📊 Distribution de la Confiance par Sentiment'
)

fig.update_layout(
    xaxis_title='Sentiment',
    yaxis_title='Confiance',
    yaxis=dict(tickformat='.0%'),
    showlegend=False
)

fig.show()

In [ ]:
# Statistiques de confiance par sentiment
confidence_stats = df_test.groupby('sentiment')['confidence'].agg(['mean', 'std', 'min', 'max'])
confidence_stats.columns = ['Moyenne', 'Écart-type', 'Min', 'Max']
confidence_stats = confidence_stats.round(3)

print("📈 Statistiques de Confiance par Sentiment :")
display(confidence_stats)

## 7. Analyse de la Longueur des Textes

In [ ]:
# Ajouter la longueur du texte
df_test['text_length'] = df_test['text'].str.len()
df_test['word_count'] = df_test['text'].str.split().str.len()

# Scatter plot : longueur vs confiance
fig = px.scatter(
    df_test,
    x='text_length',
    y='confidence',
    color='sentiment',
    color_discrete_map=colors,
    size='word_count',
    hover_data=['text'],
    title='🔍 Relation Longueur du Texte vs Confiance'
)

fig.update_layout(
    xaxis_title='Longueur du texte (caractères)',
    yaxis_title='Confiance',
    yaxis=dict(tickformat='.0%')
)

fig.show()

## 8. Test de Performances du Modèle

In [ ]:
import time

# Mesurer le temps d'analyse
test_texts = [
    "Court texte.",
    "Ceci est un texte de longueur moyenne pour tester les performances.",
    "Ceci est un texte beaucoup plus long qui contient de nombreux mots et devrait prendre plus de temps à analyser car le modèle BERT doit traiter plus de tokens. Nous voulons voir comment le temps d'analyse évolue avec la longueur du texte."
]

performance_results = []

for text in test_texts:
    start_time = time.time()
    result = analyzer.analyze(text)
    elapsed_time = time.time() - start_time
    
    performance_results.append({
        'text_length': len(text),
        'word_count': len(text.split()),
        'time_ms': elapsed_time * 1000,
        'sentiment': result['sentiment'],
        'confidence': result['confidence']
    })

df_perf = pd.DataFrame(performance_results)
print("⏱️ Performances d'analyse :")
display(df_perf)

In [ ]:
# Graphique de performance
fig = px.bar(
    df_perf,
    x='word_count',
    y='time_ms',
    color='sentiment',
    color_discrete_map=colors,
    title='⏱️ Temps d\'Analyse vs Nombre de Mots',
    labels={'word_count': 'Nombre de mots', 'time_ms': 'Temps (ms)'}
)

fig.show()

## 9. Matrice de Confusion (Simulation)

In [ ]:
# Simuler des prédictions attendues vs réelles
expected_sentiments = [
    'très positif', 'positif', 'très positif', 'positif', 'positif',
    'négatif', 'négatif', 'négatif', 'très négatif', 'négatif',
    'neutre', 'neutre', 'neutre', 'neutre', 'neutre'
]

# Comparer avec les prédictions
df_test['expected'] = expected_sentiments
df_test['correct'] = df_test['sentiment'] == df_test['expected']

accuracy = df_test['correct'].mean()
print(f"📊 Précision globale : {accuracy:.1%}")

In [ ]:
# Créer la matrice de confusion
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Simplifier en 3 catégories
def simplify_sentiment(s):
    if 'positif' in s.lower():
        return 'positif'
    elif 'négatif' in s.lower():
        return 'négatif'
    else:
        return 'neutre'

df_test['simple_expected'] = df_test['expected'].apply(simplify_sentiment)
df_test['simple_predicted'] = df_test['sentiment'].apply(simplify_sentiment)

# Matrice de confusion
labels = ['positif', 'neutre', 'négatif']
cm = confusion_matrix(
    df_test['simple_expected'],
    df_test['simple_predicted'],
    labels=labels
)

# Visualiser avec plotly
fig = px.imshow(
    cm,
    labels=dict(x="Prédit", y="Réel", color="Nombre"),
    x=labels,
    y=labels,
    title='🎯 Matrice de Confusion (Simplifiée)',
    color_continuous_scale='Blues',
    text_auto=True
)

fig.show()

## 10. Simulation d'Évolution d'Humeur

In [ ]:
# Simuler une semaine de données
import random

# Générer des données sur 7 jours
simulation_data = []
base_date = datetime.now() - timedelta(days=7)

sentiments_weights = {
    0: 0.1,  # très négatif
    1: 0.2,  # négatif
    2: 0.3,  # neutre
    3: 0.3,  # positif
    4: 0.1   # très positif
}

sentiment_names = ['très négatif', 'négatif', 'neutre', 'positif', 'très positif']

for day in range(7):
    # 3-5 messages par jour
    num_messages = random.randint(3, 5)
    
    for msg in range(num_messages):
        # Tendance : amélioration progressive
        day_bias = day * 0.1  # Plus le jour avance, plus c'est positif
        
        # Choisir un sentiment avec biais
        weights = [w + day_bias if i >= 2 else w - day_bias/2 
                   for i, w in sentiments_weights.items()]
        weights = [max(0.01, w) for w in weights]
        weights = [w/sum(weights) for w in weights]
        
        sentiment_idx = random.choices(range(5), weights=weights)[0]
        sentiment = sentiment_names[sentiment_idx]
        
        # Score de confiance
        confidence = random.uniform(0.5, 0.95)
        
        # Timestamp
        timestamp = base_date + timedelta(
            days=day,
            hours=random.randint(8, 22),
            minutes=random.randint(0, 59)
        )
        
        simulation_data.append({
            'timestamp': timestamp,
            'day': day + 1,
            'sentiment': sentiment,
            'sentiment_score': sentiment_idx / 4,  # 0 à 1
            'confidence': confidence
        })

df_sim = pd.DataFrame(simulation_data)
print(f"✅ {len(df_sim)} messages simulés sur 7 jours")
display(df_sim.head(10))

In [ ]:
# Visualiser l'évolution
fig = px.line(
    df_sim,
    x='timestamp',
    y='sentiment_score',
    color='sentiment',
    color_discrete_map=colors,
    title='📈 Évolution de l\'Humeur sur 7 Jours',
    markers=True
)

# Ajouter une ligne de tendance
fig.add_trace(go.Scatter(
    x=df_sim['timestamp'],
    y=df_sim['sentiment_score'].rolling(window=3, min_periods=1).mean(),
    mode='lines',
    name='Tendance (moyenne mobile)',
    line=dict(color='black', width=2, dash='dash')
))

fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Score d\'humeur',
    yaxis=dict(tickformat='.0%', range=[0, 1])
)

fig.show()

In [ ]:
# Moyenne par jour
daily_avg = df_sim.groupby('day').agg({
    'sentiment_score': 'mean',
    'confidence': 'mean'
}).reset_index()

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Score d\'Humeur Moyen par Jour', 'Confiance Moyenne par Jour')
)

# Score d'humeur
fig.add_trace(
    go.Bar(
        x=daily_avg['day'],
        y=daily_avg['sentiment_score'],
        marker_color='lightseagreen',
        name='Score humeur'
    ),
    row=1, col=1
)

# Confiance
fig.add_trace(
    go.Bar(
        x=daily_avg['day'],
        y=daily_avg['confidence'],
        marker_color='coral',
        name='Confiance'
    ),
    row=1, col=2
)

fig.update_layout(
    title_text='📊 Statistiques Journalières',
    showlegend=False,
    height=400
)

fig.show()

## 11. Résumé et Conclusions

In [ ]:
# Résumé final
print("="*60)
print("📊 RÉSUMÉ DE L'EXPLORATION")
print("="*60)

print(f"\n📝 Données analysées :")
print(f"   • Phrases de test : {len(df_test)}")
print(f"   • Simulation 7 jours : {len(df_sim)} messages")

print(f"\n🎯 Performances du modèle :")
print(f"   • Précision (test) : {accuracy:.1%}")
print(f"   • Confiance moyenne : {df_test['confidence'].mean():.1%}")

print(f"\n📈 Tendances (simulation) :")
start_score = daily_avg['sentiment_score'].iloc[0]
end_score = daily_avg['sentiment_score'].iloc[-1]
improvement = (end_score - start_score) / start_score * 100
print(f"   • Score début : {start_score:.1%}")
print(f"   • Score fin : {end_score:.1%}")
print(f"   • Amélioration : {improvement:+.1f}%")

print(f"\n✅ Exploration terminée !")

---

## 📝 Notes

Ce notebook montre :
1. Comment analyser les performances du modèle BERT
2. La distribution des sentiments détectés
3. L'évolution de l'humeur dans le temps
4. Les métriques de confiance

**Prochaines étapes :**
- Collecter plus de données réelles
- Comparer avec l'Approche 2 (modèle custom)
- Analyser les cas d'erreur

---

*Projet Chatbot Bien-être - ENSA Berrechid*